In [46]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
%matplotlib inline

In [47]:
data_dir='.../input/gtsrb-german-traffic-sign'
train='../input/gtsrb-german-traffic-sign/Train'
test='../input/gtsrb-german-traffic-sign/'
height=30
width=30

In [48]:
classes=len(os.listdir(train))
classes

43

In [49]:
def data_loader(data_dir):
    images=list()
    labels=list()
    for i in range(classes):
        img_df = os.path.join(data_dir, str(i))
        for img in os.listdir(img_df):
            img = load_img(os.path.join(img_df, img),target_size=(30, 30))
            image = img_to_array(img)
            images.append(image)
            labels.append(i)
    
    return images, labels
           
    
    

In [50]:
images,labels=data_loader(train)

In [51]:
labels = to_categorical(labels)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(np.array(images),labels,test_size=0.4,random_state=42)

In [62]:

from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(height,width,3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

# Second Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Dropout(rate=0.25))

# Third Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 2, 2, 64)         

In [63]:
# Flattening the layer and adding Dense Layer
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(classes, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 2, 2, 64)         

In [64]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [66]:
# Fitting the model
EPOCHS = 35
history = model.fit(x_train, 
                    y_train,
                    validation_data = (x_test, y_test), 
                    epochs=EPOCHS, 
                    steps_per_epoch=60
                   )

Epoch 1/35
60/60 [==============================] - 2s 27ms/step - loss: 0.1368 - accuracy: 0.9610 - val_loss: 0.0608 - val_accuracy: 0.9855
Epoch 2/35
60/60 [==============================] - 1s 25ms/step - loss: 0.1319 - accuracy: 0.9601 - val_loss: 0.0643 - val_accuracy: 0.9846
Epoch 3/35
60/60 [==============================] - 2s 29ms/step - loss: 0.1226 - accuracy: 0.9641 - val_loss: 0.0644 - val_accuracy: 0.9855
Epoch 4/35
60/60 [==============================] - 2s 33ms/step - loss: 0.1237 - accuracy: 0.9649 - val_loss: 0.0615 - val_accuracy: 0.9850
Epoch 5/35
60/60 [==============================] - 1s 25ms/step - loss: 0.1231 - accuracy: 0.9634 - val_loss: 0.0685 - val_accuracy: 0.9828
Epoch 6/35
60/60 [==============================] - 2s 26ms/step - loss: 0.1199 - accuracy: 0.9638 - val_loss: 0.0613 - val_accuracy: 0.9841
Epoch 7/35
60/60 [==============================] - 2s 30ms/step - loss: 0.1178 - accuracy: 0.9652 - val_loss: 0.0561 - val_accuracy: 0.9865
Epoch 8/35
60

In [67]:
accuracy = model.evaluate(x_test, y_test)

491/491 [==============================] - 1s 3ms/step - loss: 0.0461 - accuracy: 0.9888


In [78]:
from sklearn.metrics import accuracy_score
import pandas as pd

Y_test = pd.read_csv(test + 'Test.csv')
test_labels = Y_test["ClassId"].values
Y_test = Y_test["Path"].values

output = list()
for img in Y_test:
    image = load_img(os.path.join(test,img), target_size=(30, 30))
    output.append(np.array(image))

X_test=np.array(output)
pred = model.predict_classes(X_test)

#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(test_labels, pred)*100)

Test Data accuracy:  93.83214568487728
